In [1]:
# --- GÜNCELLENMİŞ KURULUM KODU ---
# Tüm kütüphaneleri tek bir komutla kurarak pip'in bağımlılıkları daha iyi çözmesine yardımcı oluyoruz.
# Bu yöntem, versiyon çakışmalarını en aza indirir.

print(">>> Adım 1.1: Gerekli kütüphaneler tek seferde yükleniyor...")

!pip install -qU \
    google-generativeai \
    langchain-google-genai \
    langchain \
    datasets \
    sentence-transformers \
    faiss-cpu \
    langchain_community

print("\n✅ Kütüphane kurulumları tamamlandı!")
print("Not: Kurulum sırasında hala bazı uyarılar görebilirsin, bu Colab ortamında normaldir.")
print("Bir sonraki API anahtarı yükleme adımına geçebilirsin.")

>>> Adım 1.1: Gerekli kütüphaneler tek seferde yükleniyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which

In [2]:
# Bu hücre, Colab'ın güvenli 'Secrets' bölümünden API anahtarımızı alır ve
# Google'ın kütüphanelerini bu anahtarla çalışacak şekilde yapılandırır.

print(">>> Adım 1.2: Google API anahtarı yükleniyor...")

import google.generativeai as genai
from google.colab import userdata # Colab'ın 'Secrets' bölümüne erişmemizi sağlayan araç

# 'try...except' bloğu, olası bir hatayı yakalamamızı sağlar.
# Eğer anahtar doğru yüklenirse 'try' içindeki kod çalışır.
# Bir sorun olursa 'except' içindeki hata mesajı gösterilir.
try:
    # 'userdata.get()' fonksiyonu ile 'GOOGLE_API_KEY' isimli gizli anahtarımızı çağırıyoruz.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    # 'genai.configure()' fonksiyonu ile Google kütüphanelerine anahtarımızı tanıtıyoruz.
    genai.configure(api_key=GOOGLE_API_KEY)

    print("✅ Google API anahtarı başarıyla yüklendi ve yapılandırıldı.")
    print("Artık projemizin bir sonraki adımına geçmeye hazırız!")

except Exception as e:
    print("🚨 HATA: API anahtarı yüklenemedi!")
    print("Lütfen Colab'ın sol menüsündeki 'Secrets' (anahtar simgesi) bölümüne")
    print("API anahtarınızı 'GOOGLE_API_KEY' adıyla doğru bir şekilde eklediğinizden emin olun.")

>>> Adım 1.2: Google API anahtarı yükleniyor...
✅ Google API anahtarı başarıyla yüklendi ve yapılandırıldı.
Artık projemizin bir sonraki adımına geçmeye hazırız!


In [8]:
# --- Adım 2 (Genişletilmiş Alt Küme Versiyonu) ---

print(">>> Adım 2: Türkçe Wikipedia veri seti yükleniyor...")

from datasets import load_dataset

try:
    dataset = load_dataset("wikimedia/wikipedia", "20231101.tr", split='train')
    print(f"\n✅ Orijinal veri setinde toplam {len(dataset)} adet makale bulundu.")

    # --- DEĞİŞİKLİK BURADA ---
    # Kısıtlamayı 1000'den 20,000 makaleye çıkarıyoruz.
    subset_dataset = dataset.select(range(20000))

    print(f"\n✅ Geliştirme için {len(subset_dataset)} makalelik bir alt küme oluşturuldu.")
    print("Artık bu alt küme ile çalışacağız.")

    print("\n--- Örnek Veri ---")
    print("İlk makalenin başlığı:", subset_dataset[0]['title'])

except Exception as e:
    print(f"🚨 HATA: Veri seti yüklenirken bir sorun oluştu!\n{e}")

>>> Adım 2: Türkçe Wikipedia veri seti yükleniyor...

✅ Orijinal veri setinde toplam 534988 adet makale bulundu.

✅ Geliştirme için 20000 makalelik bir alt küme oluşturuldu.
Artık bu alt küme ile çalışacağız.

--- Örnek Veri ---
İlk makalenin başlığı: Cengiz Han


In [9]:
# Bu hücre, bir önceki adımda yüklediğimiz 1000 makaleyi
# daha küçük ve yönetilebilir metin parçalarına ("chunk") böler.

print(">>> Adım 3: Metinleri daha küçük parçalara ayırma işlemi başlıyor...")

# LangChain'in metin bölme aracını projemize dahil ediyoruz.
from langchain.text_splitter import RecursiveCharacterTextSplitter

try:
    # Metinleri LangChain'in anlayacağı özel bir "Document" formatına dönüştürüyoruz.
    # Her makalenin 'text' içeriğini alıp bir liste oluşturuyoruz.
    docs_list = [article['text'] for article in subset_dataset]

    # Metin bölücümüzü yapılandırıyoruz:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,      # Her bir parçanın maksimum karakter sayısı.
        chunk_overlap=150,    # Parçalar arası anlamsal bütünlüğü korumak için ortak karakter sayısı.
        length_function=len,  # Karakter sayısını nasıl ölçeceğimizi belirtiyoruz (standart Python len fonksiyonu).
        is_separator_regex=False,
    )

    # Bölücüyü kullanarak metin listemizi parçalara ayırıyoruz.
    # Bu işlem sonucunda elimizde bir "Document" nesneleri listesi olacak.
    chunks = text_splitter.create_documents(docs_list)

    print("\n✅ Metin parçalama işlemi başarıyla tamamlandı.")
    print(f"Başlangıçta {len(docs_list)} adet tam makalemiz vardı.")
    print(f"Parçalama sonrası toplam {len(chunks)} adet metin parçası (chunk) oluşturuldu.")

    # Oluşturduğumuz parçalardan birine göz atalım.
    print("\n--- Örnek Parça (Chunk) ---")
    # chunks listesindeki 10. parçanın içeriğini (page_content) yazdıralım.
    print(chunks[10].page_content)
    print("-----------------------------")

except Exception as e:
    print("🚨 HATA: Metinler parçalanırken bir sorun oluştu!")
    print(e)

>>> Adım 3: Metinleri daha küçük parçalara ayırma işlemi başlıyor...

✅ Metin parçalama işlemi başarıyla tamamlandı.
Başlangıçta 20000 adet tam makalemiz vardı.
Parçalama sonrası toplam 144739 adet metin parçası (chunk) oluşturuldu.

--- Örnek Parça (Chunk) ---
Kabul Han ve onun halefi Ambakay Han zamanında Moğollar, Çin'deki Jin İmparatorluğu ile mücadele edecek kadar kuvvetlenseler de Tatarlar, Çinlileri hoşnut etmek için Ambakay’ı Çin’e teslim ettiler. Ambakay hiç alışılmadık bir şekilde, “tahta eşek şekli” denen bir duruma sokularak çarmıha gerilip infaz edildi. Cengiz Han’ın büyük amcası Kutua, bu hakarete Çin üzerine ve Tatarlara bir dizi saldırı düzenleyerek cevap verdi ve bu akınlar sonunda “Moğol Herkülü” unvanını kazandı. Fakat, 1160 yılında, detayları bilinmeyen bir dizi olay sonunda, Kuzey Çin’in hakimi Jin Hanedanı, Moğolları hezimete uğrattı. Moğollar bir süre karmaşa içinde dağıldılar. Sefalet içinde yüzen bu karmaşık hâldeki Moğollar’ın içerisindeki önemsiz liderlerden 

In [10]:
# Bu hücre, 10,685 adet metin parçasının TAMAMINI anlamsal vektörlere dönüştürür ve
# bu vektörleri hızlı arama için bir veritabanına kaydeder.

print(">>> Adım 4 (Tam Sürüm): Vektör veritabanı oluşturuluyor...")
print(f"⚠️ UYARI: Toplam {len(chunks)} adet metin parçası işlenecek. Bu işlem çok uzun sürebilir!")

# Gerekli araçları LangChain'den çağırıyoruz.
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

try:
    # "Anlam Tercümanımızı" (Embedding Model) yüklüyoruz.
    # Model daha önce indirildiği için bu adım hızlı olacaktır.
    model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
    print("\n✅ Embedding modeli başarıyla yüklendi.")


    # --- ANA DEĞİŞİKLİK BURADA ---
    # Artık verinin küçük bir kısmını değil, 'chunks' listesinin tamamını kullanıyoruz.
    print("Metin parçaları vektörlere dönüştürülüyor ve veritabanına ekleniyor...")
    vector_store = FAISS.from_documents(chunks, embeddings_model) # 'small_chunks' yerine 'chunks' kullanılıyor.

    print("\n✅ Vektör veritabanı başarıyla oluşturuldu!")
    print(f"Veritabanında toplam {vector_store.index.ntotal} adet vektör (metin parçası) bulunmaktadır.")

except Exception as e:
    print("🚨 HATA: Vektör veritabanı oluşturulurken bir sorun oluştu!")
    print(e)

>>> Adım 4 (Tam Sürüm): Vektör veritabanı oluşturuluyor...
⚠️ UYARI: Toplam 144739 adet metin parçası işlenecek. Bu işlem çok uzun sürebilir!

✅ Embedding modeli başarıyla yüklendi.
Metin parçaları vektörlere dönüştürülüyor ve veritabanına ekleniyor...

✅ Vektör veritabanı başarıyla oluşturuldu!
Veritabanında toplam 144739 adet vektör (metin parçası) bulunmaktadır.


In [11]:
# Bu hücre, RAG mimarisinin tüm parçalarını birleştirir ve
# kullanıcının sorduğu bir soruya, vektör veritabanındaki bilgileri
# kullanarak cevap üretir.

print(">>> Adım 5 (Düzeltilmiş Versiyon): RAG Zinciri oluşturuluyor...")

# Gerekli araçları LangChain ve Google kütüphanelerinden çağırıyoruz.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

try:
    # 1. BEYNİ (LLM) TANIMLAMA

    # --- ANA DEĞİŞİKLİK BURADA ---
    # Gemini modelini çağırırken, kimlik doğrulama hatasını önlemek için
    # API anahtarımızı 'google_api_key' parametresi ile doğrudan belirtiyoruz.
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                                 google_api_key=GOOGLE_API_KEY,
                                 temperature=0.3)

    # 2. HAFIZAYI (RETRIEVER) TANIMLAMA
    retriever = vector_store.as_retriever()

    # 3. BEYNE TALİMAT VERME (PROMPT ŞABLONU OLUŞTURMA)
    prompt_template = """
    Sana verilen aşağıdaki bağlamı (context) kullanarak soruya cevap ver.
    Cevabı, sadece bu bağlamdaki bilgilerden yola çıkarak oluştur.
    Eğer bağlamda cevap yoksa, "Bu konuda bilgim yok." de.

    Bağlam (Context):
    {context}

    Soru: {input}

    Cevap:
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    # 4. ZİNCİRİ OLUŞTURMA
    combine_docs_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

    print("\n✅ RAG Zinciri başarıyla oluşturuldu!")

    # 5. İLK SORUMUZU SORMA
    test_sorusu = "türkiye nin başkenti neresi?"
    print(f"\n❓ Soru soruluyor: '{test_sorusu}'")

    response = retrieval_chain.invoke({"input": test_sorusu})

    print("\n🤖 Gelen Cevap:")
    print(response["answer"])

except Exception as e:
    print("🚨 HATA: RAG zinciri çalıştırılırken bir sorun oluştu!")
    print(e)

>>> Adım 5 (Düzeltilmiş Versiyon): RAG Zinciri oluşturuluyor...

✅ RAG Zinciri başarıyla oluşturuldu!

❓ Soru soruluyor: 'türkiye nin başkenti neresi?'

🤖 Gelen Cevap:
Ankara, Türkiye'nin başkentidir.
